In [30]:
import yfinance as yf
import yfinance.shared as shared
import time
import pandas as pd
# from datetime import date, timedelta, datetime
from myUtils import pickle_dump, pickle_load, read_symbols_file # NOQA
from myUtils import drop_symbols_all_NaN, chunked_list # NOQA
from myUtils import yf_download_AdjOHLCV_noAutoAdj
from yf_utils import _2_split_train_val_test, _3_random_slices, _4_lookback_slices, _5_perf_ranks
from yf_utils import _6_grp_tuples_sort_sum

verbose = False  # True prints more output

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_df_c = 'df_close_clean' 

In [31]:
df_c = pickle_load(path_data_dump, filename_pickled_df_c)
df_train, df_val, df_test = _2_split_train_val_test(df_c)
len_df_train = len(df_train)
len_df_val = len(df_val)
len_df_test = len(df_test)

In [32]:
n_samples = 3  # number of tuples to create for iloc start_train:end_train:end_eval
# n_samples = 200  # number of tuples to create for iloc start_train:end_train:end_eval
days_lookbacks = [30, 120, 60]  # number of lookback days for training
days_eval = 10  # number of days to evaluate effectiveness of the training
set_top_syms = 5  # number of the most-common symbols from days_lookbacks' performance rankings

In [33]:
# create n_samples of iloc start_train:end_train:end_eval using max value in days_loobacks
max_lookback_slices = _3_random_slices(len_df_train, n_samples=n_samples, days_lookback=max(days_lookbacks), days_eval=days_eval)
sets_lookback_slices = _4_lookback_slices(max_slices=max_lookback_slices, days_lookbacks=days_lookbacks, verbose=False)
print(f'max_lookback_slices:\n{max_lookback_slices}')
print(f'sets_lookback_slices:\n{sets_lookback_slices}')

max_lookback_slices:
[(878, 998, 1008), (466, 586, 596), (578, 698, 708)]
sets_lookback_slices:
[[(968, 998, 1008), (938, 998, 1008), (878, 998, 1008)], [(556, 586, 596), (526, 586, 596), (466, 586, 596)], [(668, 698, 708), (638, 698, 708), (578, 698, 708)]]


In [34]:
grp_top_set_syms_n_freq = []  # list of lists of top_set_symbols_n_freq, there are n_samples lists in list
grp_top_set_syms = []  # list of lists of top_set_symbols, there are n_samples lists in list
# loop thru lists of tuples of start_train:end_train:end_eval, i.e.
#  [[(887, 917, 927), (857, 917, 927), (797, 917, 927)],
#  [(483, 513, 523), (453, 513, 523), (393, 513, 523)]]
for lb_slices in sets_lookback_slices:
  grp_most_common_syms = []  
  for lb_slice in lb_slices:  # lb_slice, e.g. (246, 276, 286)
    start_train = lb_slice[0]
    end_train = lb_slice[1]
    start_eval = end_train
    end_eval = lb_slice[2]
    lookback = end_train - start_train
    eval = end_eval - start_eval
    print(f'lb_slices:     {lb_slices}')
    print(f'lb_slice:      {lb_slice}')
    print(f'days lookback: {lookback}')
    print(f'days eval:     {eval}')    
    print(f'start_train:   {start_train}')
    print(f'end_train:     {end_train}')
    # print(f'start_eval:    {start_eval}')
    # print(f'end_eval:      {end_eval}')

    _df = df_train.iloc[start_train:end_train]
    perf_ranks, most_common_syms = _5_perf_ranks(_df, n_top_syms=10)
    # 1 lookback of r_CAGR/UI, r_CAGR/retnStd, r_retnStd/UI
    print(f'perf_ranks: {perf_ranks}')  
    # most common symbols of perf_ranks 
    print(f'most_common_syms: {most_common_syms}')     
    # grp_perf_ranks[lookback] = perf_ranks
    print(f'+++ finish lookback slice {lookback} +++\n')
    grp_most_common_syms.append(most_common_syms)

  print(f'grp_most_common_syms: {grp_most_common_syms}')
  # grp_most_common_syms a is list of lists of tuples of 
  #  the most-common-symbols symbol:frequency cumulated from
  #  each days_lookback  
  print(f'**** finish lookback slices {lb_slices} ****\n')
  # flatten list of lists of (symbol:frequency)
  flat_grp_most_common_syms = [val for sublist in grp_most_common_syms for val in sublist]
  # group symbols from set of days_lookbacks (i.e. lb_slices) and sum frequency of the symbols
  set_most_common_syms = _6_grp_tuples_sort_sum(flat_grp_most_common_syms, reverse=True)
  # get the top few most-frequent symbol:frequency pairs
  top_set_syms_n_freq = set_most_common_syms[0:set_top_syms]
  # get symbols from top_set_syms_n_freq
  top_set_syms = [i[0] for i in top_set_syms_n_freq]
  grp_top_set_syms_n_freq.append(top_set_syms_n_freq)
  grp_top_set_syms.append(top_set_syms)
  print(f'top {set_top_syms} ranked symbols and frequency from set {lb_slices}:\n{top_set_syms_n_freq}')
  print(f'top {set_top_syms} ranked symbols from set {lb_slices}:\n{top_set_syms}')  
  print(f'===== finish top {set_top_syms} ranked symbols from days_lookback set {lb_slices} =====\n\n')

lb_slices:     [(968, 998, 1008), (938, 998, 1008), (878, 998, 1008)]
lb_slice:      (968, 998, 1008)
days lookback: 30
days eval:     10
start_train:   968
end_train:     998
perf_ranks: {'period-30': {'r_CAGR/UI': array(['CHRD', 'MARA', 'VERU', 'EDIT', 'BEEM', 'UAN', 'ACRS', 'IPI',
       'SLCA', 'BIDU'], dtype=object), 'r_CAGR/retnStd': array(['CHRD', 'MARA', 'VERU', 'EDIT', 'BEEM', 'ACRS', 'SLCA', 'UAN',
       'MSTR', 'IPI'], dtype=object), 'r_retnStd/UI': array(['CHRD', 'BIDU', 'AJRD', 'MT', 'JOUT', 'AEHR', 'IAC', 'JHI', 'PNC',
       'NATI'], dtype=object)}}
most_common_syms: [('CHRD', 3), ('MARA', 2), ('VERU', 2), ('EDIT', 2), ('BEEM', 2), ('UAN', 2), ('ACRS', 2), ('IPI', 2), ('SLCA', 2), ('BIDU', 2), ('MSTR', 1), ('AJRD', 1), ('MT', 1), ('JOUT', 1), ('AEHR', 1), ('IAC', 1), ('JHI', 1), ('PNC', 1), ('NATI', 1)]
+++ finish lookback slice 30 +++

lb_slices:     [(968, 998, 1008), (938, 998, 1008), (878, 998, 1008)]
lb_slice:      (938, 998, 1008)
days lookback: 60
days eval:     

In [36]:
print(f'grp_top_set_syms_n_freq:\n{grp_top_set_syms_n_freq}')
print(f'grp_top_set_syms:\n{grp_top_set_syms}')

grp_top_set_syms_n_freq:
[[('CHRD', 8), ('BEEM', 6), ('MARA', 6), ('MSTR', 5), ('SLCA', 4)], [('AXSM', 7), ('DMRC', 6), ('ENPH', 6), ('FTSM', 6), ('JYNT', 6)], [('FTSM', 5), ('HOV', 5), ('TNK', 5), ('ARMK', 4), ('IRT', 4)]]
grp_top_set_syms:
[['CHRD', 'BEEM', 'MARA', 'MSTR', 'SLCA'], ['AXSM', 'DMRC', 'ENPH', 'FTSM', 'JYNT'], ['FTSM', 'HOV', 'TNK', 'ARMK', 'IRT']]
